In [1]:
import optuna
import os
from pathlib import Path
import kaggle
from catboost import CatBoostClassifier
import pandas as pd
import numpy as np
from sklearn.preprocessing import FunctionTransformer, PowerTransformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
if iskaggle:
    path = Path('../input/playground-series-s3e23')
    orig_path = Path('../input/software-defect-prediction')
else:
    path = Path('../input/playground-series-s3e23')
    orig_path = Path('../input/software-defect-prediction')
    if not path.exists():
        kaggle.api.competition_download_cli(str(path))
        zipfile.ZipFile(f'{path}.zip').extractall(path)
    if not orig_path.exists():
        kaggle.api.competition_download_cli(str(path))
        zipfile.ZipFile(f'{path}.zip').extractall(path)

SEED = 42
SPLITS = 5
VERSION = 4
OUTPUT = f'cat_boost_clf_submisson_v{VERSION}.csv'

np.random.seed(SEED)

/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train = pd.read_csv(path/'train.csv', index_col = 'id')
test = pd.read_csv(path/'test.csv', index_col = 'id')
orig_train = pd.read_csv(orig_path/'jm1.csv')

In [3]:
for object_features in list(orig_train.loc[:, orig_train.dtypes == 'O']):
    orig_train[object_features] = orig_train[object_features].replace({'?' : np.nan}).astype('float64')

In [4]:
X = pd.concat([train, orig_train])
y = X.pop('defects')

In [5]:
partial_pipeline = make_pipeline(SimpleImputer(), FunctionTransformer(func=np.log1p, validate=False), PowerTransformer())

In [6]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators',50,1000, log=True),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 0.7,log=True),
        'depth': trial.suggest_int('depth',1, 12, log=True),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg',1e-8, 100,log=True),
        'model_size_reg': trial.suggest_float('model_size_reg',1e-8,100,log=True),
        'random_strength': trial.suggest_float('random_strength',1e-8,10,log=True),
        'subsample': trial.suggest_float("subsample", 0.1, 1),
        'verbose': trial.suggest_categorical('verbose', [False])
    }
    pipeline = make_pipeline(partial_pipeline, CatBoostClassifier(**params))
    scores = cross_val_score(pipeline, X, y, cv=SPLITS, scoring='roc_auc', n_jobs=-1)
    return scores.mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30, show_progress_bar=True)

[I 2023-10-15 23:17:09,272] A new study created in memory with name: no-name-91ff6aea-0804-4529-b247-fbbb561e4f06
Best trial: 0. Best value: 0.782675:   3%|█████▍                                                                                                                                                              | 1/30 [00:09<04:23,  9.08s/it]

[I 2023-10-15 23:17:18,350] Trial 0 finished with value: 0.7826748781858139 and parameters: {'n_estimators': 111, 'learning_rate': 0.5136924487289304, 'depth': 1, 'l2_leaf_reg': 1.0257324702859163e-05, 'model_size_reg': 10.513849125410047, 'random_strength': 0.02160733546109541, 'subsample': 0.5976251147682442, 'verbose': False}. Best is trial 0 with value: 0.7826748781858139.


Best trial: 1. Best value: 0.785069:   7%|██████████▉                                                                                                                                                         | 2/30 [00:57<14:57, 32.07s/it]

[I 2023-10-15 23:18:06,515] Trial 1 finished with value: 0.7850685132464286 and parameters: {'n_estimators': 501, 'learning_rate': 0.2164311228403368, 'depth': 4, 'l2_leaf_reg': 38.20788660408033, 'model_size_reg': 0.04553418773588748, 'random_strength': 1.6928907527691608e-06, 'subsample': 0.5226838280148083, 'verbose': False}. Best is trial 1 with value: 0.7850685132464286.


Best trial: 1. Best value: 0.785069:  10%|████████████████▍                                                                                                                                                   | 3/30 [01:44<17:37, 39.17s/it]

...

[I 2023-10-15 23:35:28,500] Trial 27 finished with value: 0.7856555641870002 and parameters: {'n_estimators': 781, 'learning_rate': 0.07259319675088752, 'depth': 2, 'l2_leaf_reg': 0.15639409631405649, 'model_size_reg': 0.5332078082922016, 'random_strength': 7.922614396581367e-08, 'subsample': 0.5779107057140932, 'verbose': False}. Best is trial 20 with value: 0.7858636509474153.


Best trial: 20. Best value: 0.785864:  97%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌     | 29/30 [19:42<01:09, 69.38s/it]

[I 2023-10-15 23:36:52,243] Trial 28 finished with value: 0.7832165695027731 and parameters: {'n_estimators': 558, 'learning_rate': 0.04299399883266898, 'depth': 7, 'l2_leaf_reg': 0.017687755506732163, 'model_size_reg': 0.12615824538073864, 'random_strength': 3.66917851855476e-07, 'subsample': 0.7724669134773252, 'verbose': False}. Best is trial 20 with value: 0.7858636509474153.


Best trial: 20. Best value: 0.785864: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [21:20<00:00, 42.67s/it]

[I 2023-10-15 23:38:29,294] Trial 29 finished with value: 0.7455749992195538 and parameters: {'n_estimators': 878, 'learning_rate': 0.4145251922299088, 'depth': 5, 'l2_leaf_reg': 0.0019277271850693459, 'model_size_reg': 8.459567828062162e-05, 'random_strength': 1.4311978698091337e-06, 'subsample': 0.6297096366163148, 'verbose': False}. Best is trial 20 with value: 0.7858636509474153.


In [7]:
study.best_params, study.best_value

({'n_estimators': 647,
  'learning_rate': 0.062126136754221545,
  'depth': 3,
  'l2_leaf_reg': 0.1675463052495161,
  'model_size_reg': 0.003576422478697353,
  'random_strength': 4.2456958775047355e-07,
  'subsample': 0.6625232716854229,
  'verbose': False},
 0.7858636509474153)

In [8]:
pipeline = make_pipeline(partial_pipeline, CatBoostClassifier(**study.best_params))
pipeline.fit(X, y)

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('simpleimputer', SimpleImputer()),
                                 ('functiontransformer',
                                  FunctionTransformer(func=<ufunc 'log1p'>)),
                                 ('powertransformer', PowerTransformer())])),
                ('catboostclassifier',
                 <catboost.core.CatBoostClassifier object at 0x7f9e2e3b1e50>)])

In [9]:
submission = test.copy()
submission['defects'] = pipeline.predict_proba(submission)[:, 1]
submission.defects.to_csv(OUTPUT)